In [1]:
from pathlib import Path
import typing as T
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy
import xarray as xr
import iris
import iris.plot
import numpy as np
import os
import sys
import textwrap
import itertools
import ants
import warnings

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/ants/regrid/esmf.py:26: UserWarning:  No module named 'ESMF'
Proceeding without capabilities provided by ESMPy (ESMF).
  warnings.warn(msg.format(str(_ESMF_IMPORT_ERROR)))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/ants/regrid/_ugrid.py:19: UserWarning:  No module named 'ESMF'
Proceeding without capabilities provided by ESMPy (ESMF).
  warnings.warn(msg.format(str(_ESMF_IMPORT_ERROR)))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/experimental/raster.py:29: IrisDeprecation: iris.experimental.raster is deprecated since version 3.2, and will be removed in a future release. If you make use of this functionality, please contact the Iris Developers to discuss how to retain it (which may involve reversing the deprecation).
  warn_deprecated(wmsg)


In [2]:
PATH = Path('/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um')

In [23]:
files = PATH.glob('*.nc')

In [5]:
for file in files:
    print (file)

/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverd012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pvera012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverc012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pb000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pb012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverb012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverc000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pvera000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_p

In [6]:
data = xr.open_dataset(file)

In [17]:
data

<xarray.Dataset> Size: 40MB
Dimensions:                     (grid_longitude_t: 1466, grid_latitude_t: 976,
                                 bounds2: 2, TS0_0: 1)
Coordinates:
  * grid_longitude_t            (grid_longitude_t) float64 12kB 100.0 ... 166.0
    longitude_t                 (grid_latitude_t, grid_longitude_t) float64 11MB ...
  * grid_latitude_t             (grid_latitude_t) float64 8kB -51.91 ... -8.035
    latitude_t                  (grid_latitude_t, grid_longitude_t) float64 11MB ...
  * TS0_0                       (TS0_0) datetime64[ns] 8B 2022-02-27
Dimensions without coordinates: bounds2
Data variables:
    rotated_latitude_longitude  |S1 1B ...
    grid_longitude_t_bounds     (grid_longitude_t, bounds2) float64 23kB ...
    grid_latitude_t_bounds      (grid_latitude_t, bounds2) float64 16kB ...
    STASH_m01s00i030            (TS0_0, grid_latitude_t, grid_longitude_t) int32 6MB ...
    STASH_m01s00i033            (TS0_0, grid_latitude_t, grid_longitude_t) float64 11MB ...
Attributes:
    Conventions:  CF-1.6
    source:       Met Office Unified Model v13.0

In [21]:
vars=data.variables

In [22]:
[ v for v in vars ]

['rotated_latitude_longitude',
 'grid_longitude_t',
 'longitude_t',
 'grid_longitude_t_bounds',
 'grid_latitude_t',
 'latitude_t',
 'grid_latitude_t_bounds',
 'TS0_0',
 'STASH_m01s00i030',
 'STASH_m01s00i033']

In [36]:
files = PATH.glob('*.nc')
stashes = []
for file in files:
    data = xr.open_dataset(file)
    vars=data.variables
    for v in vars:
        if ('latitude' not in v) and ('longitude' not in v):
            print (f'{file.name} {v}')
            stashes.append(v)

umnsaa_pverd012.nc VERPLEV
umnsaa_pverd012.nc VT3HR
umnsaa_pverd012.nc STASH_m01s16i202_2
umnsaa_pverd012.nc STASH_m01s16i203_2
umnsaa_pverd012.nc STASH_m01s16i204_2
umnsaa_pvera012.nc VT1HR
umnsaa_pvera012.nc STASH_m01s00i023
umnsaa_pvera012.nc STASH_m01s00i024_2
umnsaa_pvera012.nc VT3HR
umnsaa_pvera012.nc STASH_m01s00i025
umnsaa_pvera012.nc STASH_m01s00i031
umnsaa_pvera012.nc STASH_m01s00i409
umnsaa_pvera012.nc STASH_m01s01i202_2
umnsaa_pvera012.nc STASH_m01s01i205_2
umnsaa_pvera012.nc STASH_m01s01i207_2
umnsaa_pvera012.nc VT1HR_rad_diag
umnsaa_pvera012.nc STASH_m01s01i208_2
umnsaa_pvera012.nc STASH_m01s01i235_2
umnsaa_pvera012.nc STASH_m01s02i201_2
umnsaa_pvera012.nc STASH_m01s02i205_2
umnsaa_pvera012.nc STASH_m01s02i207_2
umnsaa_pvera012.nc STASH_m01s03i217
umnsaa_pvera012.nc height_10m
umnsaa_pvera012.nc STASH_m01s03i225_2
umnsaa_pvera012.nc STASH_m01s03i226_2
umnsaa_pvera012.nc STASH_m01s03i234
umnsaa_pvera012.nc height_1_5m
umnsaa_pvera012.nc STASH_m01s03i236_2
umnsaa_pvera012.n

In [38]:
unique_stash = (list(set(stashes)))
print (unique_stash)

['STASH_m01s16i203_2', 'STASH_m01s04i201', 'STASH_m01s04i112', 'VATM_SOIL_model_level_number', 'STASH_m01s09i203_3', 'STASH_m01s03i310', 'VT1HRMAX_bounds', 'STASH_m01s01i235_2', 'STASH_m01s02i207_2', 'STASH_m01s09i218', 'VTS1', 'STASH_m01s03i236_2', 'STASH_m01s09i217', 'STASH_m01s00i031', 'STASH_m01s09i203_2', 'STASH_m01s04i204', 'STASH_m01s04i202', 'STASH_m01s08i225_2', 'STASH_m01s03i225_2', 'STASH_m01s15i202_2', 'STASH_m01s02i207', 'VT1HR_MN_bounds', 'STASH_m01s03i245_2', 'STASH_m01s30i405_2', 'STASH_m01s09i218_2', 'STASH_m01s03i245', 'STASH_m01s30i404_2', 'STASH_m01s09i204', 'STASH_m01s16i222', 'STASH_m01s03i250', 'VT1HR', 'STASH_m01s09i233_2', 'VZ_8km', 'STASH_m01s16i202_2', 'STASH_m01s30i405', 'STASH_m01s30i403', 'VERPLEV', 'STASH_m01s03i217', 'STASH_m01s04i203_3', 'STASH_m01s30i461_2', 'STASH_m01s01i235', 'height_1_5m', 'STASH_m01s04i112_2', 'STASH_m01s04i204_3', 't1min', 'STASH_m01s09i205_3', 'STASH_m01s04i204_2', 'STASH_m01s01i202', 'STASH_m01s16i202', 'height_10m', 'STASH_m01s

In [39]:
len(unique_stash)

128